# Demo Script for testing new platform

## Make sweep motion (220531)

In [1]:
from indy_utils import indydcp_client as client

import json
import time
import threading
import numpy as np

robot_ip = "192.168.1.6"  # Robot (Indy) IP
robot_name = "NRMK-Indy7"  # Robot name (Indy7)

# Create class object
indy = client.IndyDCPClient(robot_ip, robot_name)

In [2]:
# Connect to robot
indy.connect()

Connect: Server IP (192.168.1.6)


True

In [3]:
def wait_motion(period=1e-1):
    while not indy.get_robot_status()['movedone']:
        time.sleep(period)
        
def joint_move_make_sure(Q, N_repeat=2):
    for _ in range(N_repeat):
        indy.joint_move_to(Q)
        wait_motion()

In [4]:
pos_home = list(np.array([-1.,  21., -55.,  -3., -89., -88.]))
# pos_wp1 = list(np.array([ -0.,  -9., -55.,  -1., -52., -88.]))
pos_wp2 = list(np.array([  0., -45., -55.,  -1.,  -4., -88.]))
pos_wp3 = list(np.array([  -3.,   15., -126.,   -2.,   -2.,  -89.]))
# pos_wp4 = list(np.array([ -2.,  15., -64.,  -2., -54., -89.]))
indy.joint_move_to(pos_home)

In [5]:
pos_list = []
# pos_list.append(pos_wp1)
pos_list.append(pos_wp2)
pos_list.append(pos_wp3)
# pos_list.append(pos_wp4)
pos_list.append(pos_home)

In [6]:
# j_pos = indy.get_joint_pos()
# print("j_pos(degree)", np.round(np.array(j_pos)))
# pos_list.append(j_pos)

In [7]:
# execute bed sweep motion
for i in range(len(pos_list)):
    joint_move_make_sure(pos_list[i])

In [3]:
indy.disconnect()